In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("trip_count_sql").getOrCreate()

In [3]:
data = spark.read.csv(r"C:\Users\woals\data-engineering\01-spark\data\tripdata_2020_03.csv", inferSchema = True, header = True)

In [4]:
data.show(5)

+---+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+----+----------+-------------------+-----------------+------------------+----------------+--------------+
|_c0|hvfhs_license_num|dispatching_base_num|originating_base_num|   request_datetime|  on_scene_datetime|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|trip_miles|trip_time|base_passenger_fare|tolls| bcf|sales_tax|congestion_surcharge|airport_fee|tips|driver_pay|shared_request_flag|shared_match_flag|access_a_ride_flag|wav_request_flag|wav_match_flag|
+---+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+--

In [5]:
data.createOrReplaceTempView("mobility_data")

In [6]:
spark.sql("select * from mobility_data limit 5").show()

+---+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+----+----------+-------------------+-----------------+------------------+----------------+--------------+
|_c0|hvfhs_license_num|dispatching_base_num|originating_base_num|   request_datetime|  on_scene_datetime|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|trip_miles|trip_time|base_passenger_fare|tolls| bcf|sales_tax|congestion_surcharge|airport_fee|tips|driver_pay|shared_request_flag|shared_match_flag|access_a_ride_flag|wav_request_flag|wav_match_flag|
+---+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+--

In [7]:
spark.sql("select pickup_date, count(*) as trips from (select split(pickup_datetime, ' ')[0] as pickup_date from mobility_data) group by pickup_date ").show()

+-----------+------+
|pickup_date| trips|
+-----------+------+
| 2020-03-02|648990|
| 2020-03-01|784260|
| 2020-03-03|697880|
| 2020-03-04|707879|
| 2020-03-05|731165|
| 2020-03-06|872012|
| 2020-03-07|886071|
| 2020-03-08|731222|
| 2020-03-09|628940|
| 2020-03-10|626474|
| 2020-03-11|628601|
| 2020-03-12|643257|
| 2020-03-13|660914|
| 2020-03-14|569397|
| 2020-03-16|391518|
| 2020-03-15|448125|
| 2020-03-19|252773|
| 2020-03-17|312298|
| 2020-03-18|269233|
| 2020-03-20|261900|
+-----------+------+
only showing top 20 rows



In [8]:
spark.sql("select pickup_date, count(*) as trips from (select split(pickup_datetime, ' ')[0] as pickup_date from mobility_data) group by pickup_date ").explain(True)

== Parsed Logical Plan ==
'Aggregate ['pickup_date], ['pickup_date, 'count(1) AS trips#363]
+- 'SubqueryAlias __auto_generated_subquery_name
   +- 'Project ['split('pickup_datetime,  )[0] AS pickup_date#362]
      +- 'UnresolvedRelation [mobility_data], [], false

== Analyzed Logical Plan ==
pickup_date: string, trips: bigint
Aggregate [pickup_date#362], [pickup_date#362, count(1) AS trips#363L]
+- SubqueryAlias __auto_generated_subquery_name
   +- Project [split(cast(pickup_datetime#23 as string),  , -1)[0] AS pickup_date#362]
      +- SubqueryAlias mobility_data
         +- View (`mobility_data`, [_c0#17,hvfhs_license_num#18,dispatching_base_num#19,originating_base_num#20,request_datetime#21,on_scene_datetime#22,pickup_datetime#23,dropoff_datetime#24,PULocationID#25,DOLocationID#26,trip_miles#27,trip_time#28,base_passenger_fare#29,tolls#30,bcf#31,sales_tax#32,congestion_surcharge#33,airport_fee#34,tips#35,driver_pay#36,shared_request_flag#37,shared_match_flag#38,access_a_ride_flag#39